# Cartpole DQN

In [2]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
#os.environ["CUDA_VISIBLE_DEVICES"] = ""
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [5]:
import random
import gym
import numpy as np
from collections import deque
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

In [6]:
env = gym.make('CartPole-v0')

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


In [7]:
state_size = env.observation_space.shape[0]
state_size

4

In [8]:
action_size = env.action_space.n
action_size

2

In [9]:
batch_size = 32

In [10]:
n_episodes = 1001 

In [11]:
output_dir = 'model_output/cartpole/'

In [12]:
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

#### Deinfe Agent

In [14]:
class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000) # double-ended queue; acts like list, but elements can be added/removed from either end
        self.gamma = 0.95 # decay or discount rate: enables agent to take into account future actions in addition to the immediate ones, but discounted at this rate
        self.epsilon = 1.0 # exploration rate: how much to act randomly; more initially than later due to epsilon decay
        self.epsilon_decay = 0.995 # decrease number of random explorations as the agent's performance (hopefully) improves over time
        self.epsilon_min = 0.01 # minimum amount of random exploration permitted
        self.learning_rate = 0.001 # rate at which NN adjusts models parameters via SGD to reduce cost 
        self.model = self._build_model() # private method 
        
    def _build_model(self):
        model = Sequential()
        model.add(Dense(24, input_dim=self.state_size, activation='relu'))
        model.add(Dense(24, activation='relu'))
        model.add(Dense(self.action_size, activation='linear')) # 2 actions, so 2 output neurons: 0 and 1 (L/R)
        model.compile(loss='mse', optimizer=Adam(lr=self.learning_rate))
        return model
    
    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done)) # list of previous experiences, enabling re-training later

    def act(self, state):
        if np.random.rand() <= self.epsilon: # if acting randomly, take random action
            return random.randrange(self.action_size)
        act_values = self.model.predict(state) # if not acting randomly, predict reward value based on current state
        return np.argmax(act_values[0]) # pick the action that will give the highest reward (i.e., go left or right?)

    def replay(self, batch_size): # method that trains NN with experiences sampled from memory
        minibatch = random.sample(self.memory, batch_size) # sample a minibatch from memory
        for state, action, reward, next_state, done in minibatch: # extract data for each minibatch sample
            target = reward # if done (boolean whether game ended or not, i.e., whether final state or not), then target = reward
            if not done: # if not done, then predict future discounted reward
                target = (reward + self.gamma * # (target) = reward + (discount rate gamma) * 
                          np.amax(self.model.predict(next_state)[0])) # (maximum target Q based on future action a')
            target_f = self.model.predict(state) # approximately map current state to future discounted reward
            target_f[0][action] = target
            self.model.fit(state, target_f, epochs=1, verbose=0) # single epoch of training with x=state, y=target_f; fit decreases loss btwn target_f and y_hat
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

    def load(self, name):
        self.model.load_weights(name)

    def save(self, name):
        self.model.save_weights(name)

#### Test in AI Gym environment

In [15]:
agent = DQNAgent(state_size, action_size) 

In [16]:
done = False
for e in range(n_episodes): # iterate over new episodes of the game
    state = env.reset() # reset state at start of each new episode of the game
    state = np.reshape(state, [1, state_size])
    
    for time in range(5000):  # time represents a frame of the game; goal is to keep pole upright as long as possible up to range, e.g., 500 or 5000 timesteps
        env.render() #Lets visualize
        action = agent.act(state) # action is either 0 or 1 (move cart left or right); decide on one or other here
        next_state, reward, done, _ = env.step(action) # agent interacts with env, gets feedback; 4 state data points, e.g., pole angle, cart position        
        reward = reward if not done else -10 # reward +1 for each additional frame with pole upright        
        next_state = np.reshape(next_state, [1, state_size])
        agent.remember(state, action, reward, next_state, done) # remember the previous timestep's state, actions, reward, etc.        
        state = next_state # set "current state" for upcoming iteration to the current next state        
        if done: # episode ends if agent drops pole or we reach timestep 5000
            print("episode: {}/{}, score: {}, e: {:.2}" # print the episode's score and agent's epsilon
                  .format(e, n_episodes, time, agent.epsilon))
            break # exit loop
    if len(agent.memory) > batch_size:
        agent.replay(batch_size) # train the agent by replaying the experiences of the episode
    if e % 50 == 0:
        agent.save(output_dir + "weights_" + '{:04d}'.format(e) + ".hdf5")         

episode: 0/1001, score: 19, e: 1.0
episode: 1/1001, score: 13, e: 1.0
episode: 2/1001, score: 10, e: 0.99
episode: 3/1001, score: 11, e: 0.99
episode: 4/1001, score: 30, e: 0.99
episode: 5/1001, score: 17, e: 0.98
episode: 6/1001, score: 19, e: 0.98
episode: 7/1001, score: 13, e: 0.97
episode: 8/1001, score: 15, e: 0.97
episode: 9/1001, score: 23, e: 0.96
episode: 10/1001, score: 28, e: 0.96
episode: 11/1001, score: 17, e: 0.95
episode: 12/1001, score: 11, e: 0.95
episode: 13/1001, score: 13, e: 0.94
episode: 14/1001, score: 11, e: 0.94
episode: 15/1001, score: 15, e: 0.93
episode: 16/1001, score: 17, e: 0.93
episode: 17/1001, score: 12, e: 0.92
episode: 18/1001, score: 19, e: 0.92
episode: 19/1001, score: 21, e: 0.91
episode: 20/1001, score: 12, e: 0.91
episode: 21/1001, score: 39, e: 0.9
episode: 22/1001, score: 29, e: 0.9
episode: 23/1001, score: 50, e: 0.9
episode: 24/1001, score: 11, e: 0.89
episode: 25/1001, score: 28, e: 0.89
episode: 26/1001, score: 21, e: 0.88
episode: 27/1001

episode: 220/1001, score: 82, e: 0.33
episode: 221/1001, score: 107, e: 0.33
episode: 222/1001, score: 81, e: 0.33
episode: 223/1001, score: 168, e: 0.33
episode: 224/1001, score: 199, e: 0.33
episode: 225/1001, score: 68, e: 0.33
episode: 226/1001, score: 29, e: 0.32
episode: 227/1001, score: 70, e: 0.32
episode: 228/1001, score: 112, e: 0.32
episode: 229/1001, score: 199, e: 0.32
episode: 230/1001, score: 85, e: 0.32
episode: 231/1001, score: 50, e: 0.32
episode: 232/1001, score: 30, e: 0.31
episode: 233/1001, score: 101, e: 0.31
episode: 234/1001, score: 145, e: 0.31
episode: 235/1001, score: 127, e: 0.31
episode: 236/1001, score: 149, e: 0.31
episode: 237/1001, score: 55, e: 0.31
episode: 238/1001, score: 102, e: 0.3
episode: 239/1001, score: 85, e: 0.3
episode: 240/1001, score: 162, e: 0.3
episode: 241/1001, score: 107, e: 0.3
episode: 242/1001, score: 119, e: 0.3
episode: 243/1001, score: 65, e: 0.3
episode: 244/1001, score: 101, e: 0.3
episode: 245/1001, score: 79, e: 0.29
episo

KeyboardInterrupt: 